<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-24 10:01:45
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.37 C
-------------------
Today PnL: -1.28 K (-0.01%)
Current PnL: -14.05 L (-9.79%)
CY Booked + Current PnL: -5.97 L (-4.16%)
-------------------
Total profit:  3.78 L
Total loss:  -17.83 L
-------------------
Total Booked + Current PnL: 21.01 L (18.03%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (5.89%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.26 L (52.7%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-223.90,H-SC,10.65,89368.0,-5948.0,5988.0,-0.38,-6.24,6.70,0.04,161.0,-0.99,0.65,76.27,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-19.08,H-LC,5.33,111664.0,7352.0,10184.0,0.36,7.05,9.12,16.81,9.0,0.72,0.81,24.23,X5K,ATH,METALS
70,SBIN,760.30,863.00,-10.56,M-LC,4.83,216452.0,14972.0,12251.0,-0.47,7.43,5.66,13.51,58.0,1.22,1.58,20.96,XY25,NTT,BANKS
4,APOLLOHOSP,6904.43,8285.00,-18.07,H-LC,6.73,111630.0,8064.0,12648.0,0.93,7.79,11.33,20.00,14.0,0.64,0.81,22.96,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,13.95,H-LC,10.04,102321.0,8888.0,13711.0,-0.61,9.51,13.40,24.18,32.0,0.65,0.75,24.25,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,MASFIN,326.60,399.50,-12.83,H-SC,16.80,98880.0,900.0,20972.0,-1.29,0.92,21.21,22.32,148.0,0.04,0.72,43.41,XR,ATH,FINANCE
38,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,2.49,150752.0,-161.0,57467.0,0.20,-0.11,38.12,37.97,87.0,-0.00,1.10,25.17,AR,ATH,AUTO


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-23.18,H-LC,0.74,201965.0,-2535.0,43948.0,0.05,-1.24,21.76,20.25,11.0,-0.06,1.47,7.39,X40N,BTT,PHARMA
10,BAJAJHFL,122.26,152.00,-4.25,H-LC,1.97,198768.0,-1738.0,50507.0,-0.88,-0.87,25.41,24.33,23.0,-0.03,1.45,14.62,X40N,BTT,FINANCE
40,HINDUNILVR,2413.81,2723.00,-12.33,H-LC,2.48,274534.0,4187.0,30446.0,0.03,1.55,11.09,12.81,16.0,0.14,2.00,14.82,XY25,NTT,FMCG
56,LTIM,5564.16,7230.20,-1.89,H-LC,2.70,189036.0,-11274.0,71248.0,-0.89,-5.63,37.69,29.94,28.0,-0.16,1.38,30.60,X200,ATH,IT
41,HINDZINC,471.44,730.22,18.37,H-LC,2.89,193858.0,-11218.0,123798.0,-0.27,-5.47,63.86,54.89,3.0,-0.09,1.41,17.31,X5K,ATH,METALS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,761.77,1065.00,-41.47,M-LC,5.29,255153.0,-21370.0,131455.0,1.85,-7.73,51.52,39.81,54.0,-0.16,1.86,22.28,XY24,NTT,AUTO
7,ATULAUTO,624.44,844.00,3375.68,M-SC,11.32,108312.0,-39056.0,90874.0,1.60,-26.50,83.90,35.16,233.0,-0.43,0.79,10.63,XY24,NTT,AUTO
45,INDIAMART,2327.09,4850.92,-46.58,H-SC,12.89,139252.0,15916.0,117849.0,1.40,12.90,84.63,108.45,118.0,0.14,1.02,38.14,AR,ATH,MISC
3,ALKYLAMINE,2347.98,4546.37,4.57,H-SC,21.00,97330.0,-3633.0,98167.0,1.37,-3.60,100.86,93.63,147.0,-0.04,0.71,44.97,SR,ATH,CHEMICALS
58,MEDANTA,1087.93,1486.00,20.34,H-SC,9.19,154176.0,27976.0,18193.0,1.25,22.17,11.80,36.59,151.0,1.54,1.13,37.46,XY24,NTT,HEALTHCARE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,MASFIN,326.60,399.50,-12.83,H-SC,16.80,98880.0,900.0,20972.0,-1.29,0.92,21.21,22.32,148.0,0.04,0.72,43.41,XR,ATH,FINANCE
94,WHIRLPOOL,1219.98,2270.00,-36.41,M-SC,9.62,107010.0,15512.0,63243.0,-1.29,16.95,59.10,86.07,207.0,0.25,0.78,54.80,XR,NTT,DURABLES
8,AWL,325.39,485.00,-67.06,H-MC,6.22,228814.0,-39633.0,171313.0,-1.07,-14.76,74.87,49.05,81.0,-0.23,1.67,17.65,XY24,NTT,FMCG
90,VALIANTORG,512.64,838.00,-338.93,H-SC,21.20,101582.0,-31704.0,116301.0,-1.05,-23.79,114.49,63.47,153.0,-0.27,0.74,70.28,XR,NTT,CHEMICALS
28,DIGITIDE,188.38,267.99,NaN,NaN,0.85,57971.0,16716.0,719.0,-1.01,40.52,1.24,42.26,197.0,23.25,0.42,36.48,XY24,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-223.90,H-SC,10.65,89368.0,-5948.0,5988.0,-0.38,-6.24,6.70,0.04,161.0,-0.99,0.65,76.27,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,47.02,M-SC,24.00,127197.0,-15282.0,15900.0,0.19,-10.73,12.50,0.44,249.0,-0.96,0.93,50.13,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-29.75,M-SC,10.44,149916.0,-21225.0,21168.0,1.10,-12.40,14.12,-0.03,194.0,-1.00,1.09,8.49,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,126.00,H-SC,8.02,148180.0,-26379.0,26406.0,-0.45,-15.11,17.82,0.02,129.0,-1.00,1.08,28.49,OX40N,NTT,PAINTS
80,TATAELXSI,7332.28,7332.0,-17.14,H-MC,1.24,87031.0,-15621.0,15613.0,-0.30,-15.22,17.94,-0.00,101.0,-1.00,0.64,32.30,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VBL,492.64,672.28,-14.07,H-LC,13.30,260876.0,-4164.0,100802.0,-0.58,-1.57,38.64,36.46,1.0,-0.04,1.90,11.41,X40N,ATH,FMCG
32,ENRIN,1377.95,3176.70,NaN,NaN,2.37,75440.0,40991.0,3976.0,0.23,118.99,5.27,130.54,2.0,10.31,0.55,15.36,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,18.37,H-LC,2.89,193858.0,-11218.0,123798.0,-0.27,-5.47,63.86,54.89,3.0,-0.09,1.41,17.31,X5K,ATH,METALS
82,TCS,3794.03,4998.00,-24.16,H-LC,11.75,246675.0,-49259.0,143170.0,-0.52,-16.65,58.04,31.73,4.0,-0.34,1.80,0.14,X40,BTT,IT
48,INFY,1461.46,2275.00,-12.78,H-LC,7.00,279455.0,17854.0,127767.0,-0.84,6.82,45.72,55.67,6.0,0.14,2.04,13.35,X40,BTT,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,0.85,57971.0,16716.0,719.0,-1.01,40.52,1.24,42.26,197.0,23.25,0.42,36.48,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,20.34,H-SC,9.19,154176.0,27976.0,18193.0,1.25,22.17,11.80,36.59,151.0,1.54,1.13,37.46,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.37,75440.0,40991.0,3976.0,0.23,118.99,5.27,130.54,2.0,10.31,0.55,15.36,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,95.22,M-MC,14.13,233830.0,8868.0,158794.0,0.74,3.94,67.91,74.53,193.0,0.06,1.71,35.76,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.64,31.10,H-SC,12.49,134253.0,8956.0,50117.0,0.59,7.15,37.33,47.15,144.0,0.18,0.98,67.96,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,20.34,H-SC,9.19,154176.0,27976.0,18193.0,1.25,22.17,11.80,36.59,151.0,1.54,1.13,37.46,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,4.76,H-SC,1.94,117684.0,15680.0,28668.0,-0.92,15.37,24.36,43.48,121.0,0.55,0.86,34.73,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-5.13,M-LC,5.64,162688.0,11743.0,97710.0,0.38,7.78,60.06,72.51,55.0,0.12,1.19,12.06,XR,NTT,IT


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.37,75440.0,40991.0,3976.0,0.23,118.99,5.27,130.54,2.0,10.31,0.55,15.36,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-36.41,M-SC,9.62,107010.0,15512.0,63243.0,-1.29,16.95,59.10,86.07,207.0,0.25,0.78,54.80,XR,NTT,DURABLES
59,NATIONALUM,189.63,247.44,-47.89,H-MC,3.87,105550.0,5046.0,25596.0,0.54,5.02,24.25,30.49,82.0,0.20,0.77,39.05,MH,ATH,METALS
53,KPIGREEN,497.21,731.64,31.10,H-SC,12.49,134253.0,8956.0,50117.0,0.59,7.15,37.33,47.15,144.0,0.18,0.98,67.96,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-46.58,H-SC,12.89,139252.0,15916.0,117849.0,1.40,12.90,84.63,108.45,118.0,0.14,1.02,38.14,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,4.81,19622.0,-29971.0,1171.0,1.23,-60.43,5.97,-58.07,201.0,-25.59,0.14,31.59,XY24,ATH,MISC
28,DIGITIDE,188.38,267.99,NaN,NaN,0.85,57971.0,16716.0,719.0,-1.01,40.52,1.24,42.26,197.0,23.25,0.42,36.48,XY24,ATH,IT
74,SIS,477.00,477.00,2317.10,M-SC,5.58,65034.0,-16056.0,16057.0,0.16,-19.80,24.69,0.00,236.0,-1.00,0.47,31.80,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.46,71674.0,-41875.0,81923.0,0.05,-36.88,114.30,35.27,266.0,-0.51,0.52,75.42,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-223.90,H-SC,10.65,89368.0,-5948.0,5988.0,-0.38,-6.24,6.70,0.04,161.0,-0.99,0.65,76.27,OX40N,NTT,MISC


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,4.81,19622.0,-29971.0,1171.0,1.23,-60.43,5.97,-58.07,201.0,-25.59,0.14,31.59,XY24,ATH,MISC
64,RELAXO,902.64,1176.00,-29.41,H-SC,15.81,63789.0,-51749.0,86740.0,0.48,-44.79,135.98,30.28,132.0,-0.60,0.47,24.62,X40N,NTT,FOOTWEAR
33,GREENPANEL,375.16,537.00,245.61,M-SC,11.09,154208.0,-28870.0,107853.0,0.32,-15.77,69.94,43.14,223.0,-0.27,1.13,45.05,XY24,NTT,MISC
58,MEDANTA,1087.93,1486.00,20.34,H-SC,9.19,154176.0,27976.0,18193.0,1.25,22.17,11.80,36.59,151.0,1.54,1.13,37.46,XY24,NTT,HEALTHCARE
28,DIGITIDE,188.38,267.99,NaN,NaN,0.85,57971.0,16716.0,719.0,-1.01,40.52,1.24,42.26,197.0,23.25,0.42,36.48,XY24,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.56
1,25,41.03
2,50,69.97


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.87
LC    34.98
MC    22.05
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.94
X40N     14.36
X40      12.46
XY25     10.86
XR        9.21
OX40N     8.43
AR        8.24
X5K       2.22
MH        1.75
X200      1.38
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.11
H-SC    25.47
H-MC    18.94
M-SC    14.97
M-LC     5.82
M-MC     2.75
L-SC     1.43
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-1.53,35.49
IT,11.86,-11.86,66.97
FINANCE,8.48,-12.20,52.91
BANKS,7.67,-8.56,52.85
MISC,6.02,-16.57,71.01
PAINTS,5.59,-14.09,36.58
ELECTRICAL,5.40,0.44,46.73
HEALTHCARE,4.50,2.20,23.78
AUTO,4.37,-12.76,61.81


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2613900.0,25
XR,1031046.0,13
AR,1015872.0,10
X40,747139.0,10
X40N,582527.0,12
XY25,452039.0,8
OX40N,318240.0,11
SR,184382.0,2
X5K,133982.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2332274.0,26
M-SC,1577799.0,20
H-LC,1329912.0,21
H-MC,1111784.0,15
M-MC,301744.0,2
M-LC,264148.0,4
L-SC,202884.0,3
L-MC,53792.0,1
L-LC,45885.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,911867.0,7
M-SC,XY24,762612.0,7
H-SC,AR,504877.0,3
H-LC,X40,484096.0,6
H-MC,XY24,433296.0,4
H-SC,XR,338744.0,4
H-LC,X40N,304625.0,6
M-MC,XY24,301744.0,2
M-SC,XR,277539.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
